In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium import webdriver
from IPython.display import clear_output
from termcolor import colored

import time
import pandas as pd

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [31]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.set_window_size(1000, 800)

columns = []
all_trades = []

def remove_campaign():
    try:
        driver.execute_script("""
            document.querySelector('.Campaign').outerHTML = '';
            document.querySelector('html').classList.remove('om-position-popup');
            document.querySelector('body').classList.remove('om-effect-overlay');
        """)
    except:
        print("[/] No campaign detected.")
    
    try:
        driver.execute_script("document.querySelector('.gated-list').outerHTML = '';")
    except:
        print("[/] No login gate detected.")

def prelims():
    driver.get("https://www.benzinga.com/sec/insider-trades/search/insider_trading_all")
    time.sleep(2)
    # driver.find_element(By.CSS_SELECTOR, ".hedrick-ButtonElement--content button").click()
    driver.execute_script("document.querySelector('.gated-list').outerHTML = ''")
    
    driver.find_element(By.CLASS_NAME, "vs-select--state-null").click()
    driver.find_element(By.CSS_SELECTOR, ".vs-select__options__content button:nth-child(5)").click()
    
    time.sleep(6)
    remove_campaign()
    time.sleep(1)
    scrape()

def next_page():
    try:
        driver.find_element(By.CSS_SELECTOR, ".vs-pagination__button.nextActive").click()
        print("[+] Navigating to next page.")
        time.sleep(1)
        remove_campaign()
    except:
        print("[-] No next page found.")

def print_progress(page, page_count, trade_num, trade_count):
    clear_output(wait=False)
    print(f"Current Page   : {str(page + 1):>{len(str(max(page_count, trade_count)))}}\u002F{str(page_count):>{len(str(max(page_count, trade_count)))}}", flush=True)
    print(f"Trade Progress : {str(trade_num + 1):>{len(str(max(page_count, trade_count)))}}\u002F{str(trade_count):>{len(str(max(page_count, trade_count)))}}", flush=True)
    
def scrape():
    for column in driver.find_elements(By.CSS_SELECTOR, ".vs-table table .vs-table__thead tr th"):
        if (column.text):
            columns.append(column.text)

    print(columns)

    page_count = int(driver.find_element(By.CSS_SELECTOR, ".vs-pagination__button:nth-last-child(1)").text)

    for p in range(page_count):
        page_trades = driver.find_elements(By.CSS_SELECTOR, ".vs-table table .vs-table__tbody tr")

        trade_count = len(page_trades)

        for t in range(trade_count):
            sing_trade = {}
            elements = page_trades[t].find_elements(By.TAG_NAME, "td")[1:-1]
            for j in range(len(columns)):
                sing_trade[columns[j]] = elements[j].text
            all_trades.append(sing_trade)
            print_progress(p, page_count, t, trade_count)

        next_page()
        time.sleep(1.5)
prelims()


Current Page   : 128/128
Trade Progress :  53/ 53
[-] No next page found.


In [32]:
pd.DataFrame(all_trades)

,Ticker,Company Name,Insider Names,Title,Qty.,Price,Size ($),Filing Date,Type,Δ Own,Own,Trade Dates
0,RMD,Resmed Inc,Robert Douglas Presi...,PRESIDENT AND COO,-1.50K,$216.27,$324.4K,"Jun 20, 2023",SELL,-0.55%,270.4K,Jun 15
1,RCG,"Renn Fund, Inc.",Murray Stahl,PRESIDENT AND CEO,420,$1.82,$764.4,"Jun 20, 2023",BUY,+0.19%,224.54K,Jun 16
2,OTLC,"Oncotelic Therapeutics, Inc.",Trieu Vuong,CHAIRMAN AND CEO,100K,$0.03,$2.6K,"Jun 20, 2023",BUY,+0.10%,98.42M,Jun 16
3,FLCX,Flooidcx Corp.,Dennis M Danzik,CEO,3K,$3.15,$9.45K,"Jun 20, 2023",BUY,+5.07%,62.22K,Jun 16
4,SNPS,Synopsys Inc,Richard Mahoney,CHIEF REVENUE OFFICE...,2.04K,$447.63,$7.78M,"Jun 20, 2023",SELL-OPTIONS,"+5,837.14%",2.08K,Jun 15 - Jun 16
...,...,...,...,...,...,...,...,...,...,...,...,...
12748,BW,"Babcock & Wilcox Enterprises, Inc.",Jimmy B Morgan,CHIEF OPERATING OFFI...,7.5K,$5.25,$39.37K,"Mar 22, 2023",BUY,+1.88%,406.53K,Mar 20
12749,BW,"Babcock & Wilcox Enterprises, Inc.",Kenneth M Young,CHIEF EXECUTIVE OFFI...,10K,$5.55,$55.48K,"Mar 22, 2023",BUY,+0.80%,1.25M,Mar 20
12750,CB,Chubb Ltd,Theodore Shasta,DIRECTOR,-1.00K,$189.73,$189.73K,"Mar 22, 2023",SELL,-6.46%,14.49K,Mar 20
12751,PFSW,Pfsweb Inc,Michael Willoughby,CEO/PRESIDENT,50K,$4.22,$211K,"Mar 22, 2023",BUY,+8.73%,623.01K,Mar 21


In [33]:
all_ins_trading = all_trades.copy()

In [38]:
all_ins_df = pd.DataFrame(all_ins_trading)

In [39]:
all_ins_df.to_csv('all_ins_trading.csv')

In [47]:
import requests
import json

req = requests.post(url="https://www.benzinga.com/sec/insider-trades/api/insider-trades", data=json.dumps({"insider_titles":[],"trade_types":["p","s"],"filing_date_preset":"last_90_days","trade_date_preset":"custom","group_by":"filing","sort_by":"last_filing_date","page":1,"page_limit":100}))